In [2]:
import os
import cv2


# Nightcall Project

Image Stiching

In [1]:
import cv2
import os

mainFolder = "Images"
myFolders = os.listdir(mainFolder)
print(myFolders)

max_width = 800  # Ancho máximo deseado
max_height = 600  # Alto máximo deseado

for folder in myFolders:
    path = mainFolder + '/' + folder
    images = []
    myList = os.listdir(path)
    print(f'Total number of images detected {len(myList)}')
    for imgN in myList:
        current_img = cv2.imread(f'{path}/{imgN}')
        images.append(current_img)
    
    sticher = cv2.Stitcher.create()
    (status, result) = sticher.stitch(images)
    
    if status == cv2.STITCHER_OK:
        print("Panorama generated")
        if result.shape[1] > max_width or result.shape[0] > max_height:
            # Si es muy grande, reducir su tamaño
            result = cv2.resize(result, (max_width, max_height))
        
        cv2.namedWindow(folder, cv2.WINDOW_NORMAL)
        cv2.imshow(folder, result)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Panorama generation unsuccessful")

cv2.destroyAllWindows()


['Ejemplo 1', 'Ejemplo 2', 'Ejemplo 3', 'Ejemplo 4']
Total number of images detected 5
Panorama generated
Total number of images detected 3
Panorama generated
Total number of images detected 4
Panorama generated
Total number of images detected 6
Panorama generated


Colorear imagenes blanco y negro

In [44]:
import numpy as np
import cv2
import os

# Directorio principal y rutas de archivos
directorio_principal = r"C:\Users\Eric\Desktop\nightcall"
prototxt_modelo = os.path.join(directorio_principal, r"model/colorization_deploy_v2.prototxt")
archivo_puntos = os.path.join(directorio_principal, r"model/pts_in_hull.npy")
caffemodel_modelo = os.path.join(directorio_principal, r"model/colorization_release_v2.caffemodel")
carpeta_imagenes = r"C:\Users\Eric\Desktop\nightcall\Images_BlackWhite"

# Cargar el modelo
print("Iniciando la carga del modelo...")
red_neuronal = cv2.dnn.readNetFromCaffe(prototxt_modelo, caffemodel_modelo)
puntos = np.load(archivo_puntos)

# Ajustar el modelo para equilibrar el canal AB
capa_clase = red_neuronal.getLayerId("class8_ab")
capa_conv = red_neuronal.getLayerId("conv8_313_rh")
puntos = puntos.transpose().reshape(2, 313, 1, 1)
red_neuronal.getLayer(capa_clase).blobs = [puntos.astype("float32")]
red_neuronal.getLayer(capa_conv).blobs = [np.full([1, 313], 2.606, dtype="float32")]

# Obtener la lista de archivos de imagen en la carpeta especificada
archivos_imagen = [os.path.join(carpeta_imagenes, archivo) for archivo in os.listdir(carpeta_imagenes) if archivo.lower().endswith(('png', 'jpg', 'jpeg'))]

for archivo_imagen in archivos_imagen:
    # Cargar la imagen en escala de grises
    print(f"Procesando la imagen: {archivo_imagen}")
    imagen = cv2.imread(archivo_imagen)
    imagen_escalada = imagen.astype("float32") / 255.0
    imagen_lab = cv2.cvtColor(imagen_escalada, cv2.COLOR_BGR2LAB)

    imagen_redimensionada = cv2.resize(imagen_lab, (224, 224))
    canal_l = cv2.split(imagen_redimensionada)[0]
    canal_l -= 50

    # Colorizar la imagen
    print("Colorizando la imagen...")
    red_neuronal.setInput(cv2.dnn.blobFromImage(canal_l))
    valores_ab = red_neuronal.forward()[0, :, :, :].transpose((1, 2, 0))
    valores_ab = cv2.resize(valores_ab, (imagen.shape[1], imagen.shape[0]))

    canal_l = cv2.split(imagen_lab)[0]
    imagen_colorizada = np.concatenate((canal_l[:, :, np.newaxis], valores_ab), axis=2)
    imagen_colorizada = cv2.cvtColor(imagen_colorizada, cv2.COLOR_LAB2BGR)
    imagen_colorizada = np.clip(imagen_colorizada, 0, 1)
    imagen_colorizada = (255 * imagen_colorizada).astype("uint8")

    # Redimensionar la imagen coloreada para mostrarla en una ventana más pequeña
    imagen_colorizada_pequena = cv2.resize(imagen_colorizada, (600, 600))
    imagen_original_pequena = cv2.resize(imagen_colorizada, (600, 600))

    # Mostrar las imágenes originales y coloreadas
    cv2.imshow("Original", cv2.resize(imagen, (600, 600)))
    cv2.imshow("Colorizada (Redimensionada)", imagen_colorizada_pequena)
    cv2.waitKey(0)

cv2.destroyAllWindows()


Cargando modelo
Procesando imagen: C:\Users\Eric\Desktop\nightcall\Images_BlackWhite\albert-einstein-1933340_1280.jpg
Colorizando la imagen
Procesando imagen: C:\Users\Eric\Desktop\nightcall\Images_BlackWhite\canteras.jpg
Colorizando la imagen
Procesando imagen: C:\Users\Eric\Desktop\nightcall\Images_BlackWhite\girasol.jpg
Colorizando la imagen
Procesando imagen: C:\Users\Eric\Desktop\nightcall\Images_BlackWhite\landing_craft_omaha_beach_normandy_d_day_war_vintage_france_military-772047.jpg
Colorizando la imagen
Procesando imagen: C:\Users\Eric\Desktop\nightcall\Images_BlackWhite\mary.jpg
Colorizando la imagen


Traductor lengua de signos.

In [23]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import load_model

# Configuración de Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Función para realizar la detección mediante Mediapipe
def detectar_mediapipe(image, model):
    # Convertir la imagen a formato RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    # Procesar la imagen con el modelo
    results = model.process(image)
    image.flags.writeable = True
    # Convertir de nuevo a formato BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# Función para dibujar los puntos clave de manera estilizada
def dibujar_puntos_clave(image, results):
    # Dibujar conexiones faciales con estilos
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Dibujar conexiones de postura con estilos
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Dibujar conexiones de la mano izquierda con estilos
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Dibujar conexiones de la mano derecha con estilos  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

# Función para visualizar las probabilidades de las acciones
def visualizar_probabilidades(res, acciones, input_frame, colores):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colores[num], -1)
        cv2.putText(output_frame, acciones[num], (0, 85+num*40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
    return output_frame


# Función para extraer puntos clave
def extraer_puntos_clave(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Cargar modelo y definir acciones
modelo = load_model('action.h5')
acciones = np.array(['hello', 'thanks', 'iloveyou'])

# Configuración para la detección en tiempo real
colores = [(52, 168, 83), (232, 17, 35), (86, 35, 232)]
secuencia = []
frase = []
predicciones = []
umbral = 0.5

# Inicializar la cámara
cap = cv2.VideoCapture(0)

# Configurar el modelo Mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Leer el fotograma de la cámara
        ret, frame = cap.read()

        # Realizar la detección mediante Mediapipe
        imagen, resultados = detectar_mediapipe(frame, holistic)
        
        # Dibujar puntos clave de manera estilizada
        dibujar_puntos_clave(imagen, resultados)
        
        # Lógica de predicción
        puntos_clave = extraer_puntos_clave(resultados)
        secuencia.append(puntos_clave)
        secuencia = secuencia[-30:]
            
        if len(secuencia) == 30:
            res = modelo.predict(np.expand_dims(secuencia, axis=0))[0]
            predicciones.append(np.argmax(res))
   
            if np.unique(predicciones[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > umbral: 
                    if len(frase) > 0: 
                        if acciones[np.argmax(res)] != frase[-1]:
                            frase.append(acciones[np.argmax(res)])
                    else:
                        frase.append(acciones[np.argmax(res)])

            if len(frase) > 5: 
                frase = frase[-5:]

            # Visualizar probabilidades
            imagen = visualizar_probabilidades(res, acciones, imagen, colores)
            
        # Actualizar la visualización de la frase en la pantalla
        cv2.rectangle(imagen, (0, 0), (640, 40), (52, 168, 83), -1)  # Cambiar el color del rectángulo
        cv2.putText(imagen, ' '.join(frase), (10, 30), 
                    cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1.5, (0, 0, 0), 3, cv2.LINE_AA)  # Cambiar la fuente, tamaño, color y grosor del texto

        
        # Mostrar el fotograma en la ventana
        cv2.imshow('OpenCV Feed', imagen)

        # Terminar el bucle al presionar 'q'
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Liberar la cámara y cerrar la ventana
    cap.release()
    cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti